In [1]:
import warnings
warnings.filterwarnings("ignore")
import tensorflow as tf
from typing import List
from blindml.frontend.config.task.task import parse_task_capsule
from blindml.frontend.reporting.wit import df_to_examples, create_feature_spec, create_feature_columns, \
    examples_to_array
from witwidget.notebook.visualization import WitConfigBuilder
from witwidget.notebook.visualization import WitWidget

In [2]:
task_file_fp = "/Users/maksim/dev_projects/blindml/tests/perovskite_task.jsonnet"
task = parse_task_capsule(task_file_fp)
task.search_for_model()
model = task.train_best_model()

INFO:  Stopping experiment maxs_experiment_30eb12a598a964e151e092ccab5336df
INFO:  Stop experiment success.
INFO:  Starting restful server...
INFO:  Successfully started Restful server!
INFO:  Setting local config...
INFO:  Successfully set local config!
INFO:  Starting experiment...


In [3]:
df = task._data_set._df
X_cols, y_col = task._data_set._X_cols, task._data_set._y_col
features_and_labels = X_cols + [y_col]
examples = df_to_examples(df)
feature_spec = create_feature_spec(df, features_and_labels)
create_feature_columns(X_cols, feature_spec);

In [4]:
y_col

'energy_above_hull (meV/atom)'

In [5]:
# takes a proto that's a list of protos
def custom_predict(examples_to_infer: List[tf.train.Example]):
    arr = examples_to_array(examples_to_infer, X_cols)
    return model.predict(arr)

In [7]:
num_datapoints = 200
tool_height_in_px = 500

# Load up the test dataset
test_examples = df_to_examples(df[0:num_datapoints])

# Setup the tool with the test examples and the trained classifier
config_builder = WitConfigBuilder(
    test_examples[:num_datapoints][X_cols], 
    feature_names=X_cols
).set_model_type('regression').set_custom_predict_fn(custom_predict)

WitWidget(config_builder, height=tool_height_in_px)


TypeError: list indices must be integers or slices, not list